## Low-to-high resolution mesh deformation transfer

We aim to solve a computer graphics/3D animation problem. We have a high resolution mesh $A$ (sculpted by a 3D artists) that is manually downsampled to a low resolution mesh $B$ with a clean mesh topology for deployment in a VR game. After inspection in the VR editor, another artists makes a few edits to the low-res mesh (without changing the topology, deformations only), resulting in mesh $C$. We now want to _transfer_ these deformations from the low-res back to the high-res mesh.

The idea is to take do so in two steps
1. Register the $B$ to $A$ by the closest-point method: for each $B$ vertex, find the closest point on $A$. Call these points the _handles_. (Generally, this point is not a vertex, but instead described by a face in $A$ and its barycentric coordinates $\lambda_{0,1,2}$ therein)
2. Since $B$ and $C$ vertices are in one-to-one correspondence, we know how the handles need to move.
3. Deform the remaining vertices of the mesh by interpolating the deformation at the handles, for example using a biharmonic deformation field with suitable boundary conditions.

In this notebook we implement the workflow in two phases:
- **Phase 1 (simpler):** vertex handles (pick a single closest vertex in $A$ for each low-res vertex).
- **Phase 2 (more accurate):** barycentric handles (use closest point on a face, then constrain that face's vertices).

Let's implement a prototype for this workflow using `numpy` and the geometry processing library `igl`.
Harmonic and biharmonic deformations in `igl` are described in [this tutorial](https://libigl.github.io/libigl-python-bindings/tut-chapter3/). Let us also keep in mind that we eventually want to implement this algorithm as a blender add-on.

We keep the code minimal and include small error summaries in printouts.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import meshplot
import igl
import copy

from jaxtyping import Float, Int
from typing import NamedTuple, Tuple

import os



In [2]:
class Mesh(NamedTuple):
    """Combine geometric and connectivity info into a single object."""
    vertices: Float[np.ndarray, "V 3"]
    faces: Int[np.ndarray, "V 3"]


In [3]:
## load meshes
v, _, _, f, _, _ = igl.readOBJ("conform_topo/coat_high.obj")
mesh_high = Mesh(vertices=v, faces=f)
v, _, _, f, _, _ = igl.readOBJ("conform_topo/coat_low_triangulated.obj")
mesh_low = Mesh(vertices=v, faces=f)
v, _, _, f, _, _ = igl.readOBJ("conform_topo/coat_low_deformed_triangulated.obj")
mesh_low_def = Mesh(vertices=v, faces=f)

  o Tenzing_HighAltitude_coat_high
  o Tenzing_HighAltitude_coat_low.001
  o Tenzing_HighAltitude_coat_low_deformed.001


In [4]:
# looks like the low res and deformed low res do NOT have the same number of vertices.
mesh_low.vertices.shape, mesh_low_def.vertices.shape

((2309, 3), (2309, 3))

In [5]:
# thus, we use some other mock data instead.

mesh_low_def = Mesh(mesh_low.vertices+np.array([1, 0, 0]),  # just a small shift
                    mesh_low.faces.copy())

In [6]:
meshplot.plot(mesh_low.vertices, mesh_low.faces)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.000846…

In [77]:
## register the low res to the high res mesh
# compute closest point on target mesh for each source vertex
distances, face_indices, points = igl.point_mesh_squared_distance(mesh_low.vertices,
                                                                  mesh_high.vertices, mesh_high.faces)

r = np.linalg.norm(mesh_low.vertices-mesh_low.vertices.mean(axis=0), axis=1).mean()

In [79]:
distances.mean()/r # very small distance to closest face

np.float64(3.054535602513566e-05)

In [80]:
np.linalg.norm(mesh_low.vertices-points, axis=1).mean() / r # distance to closest vertex. also small

np.float64(0.006597908517080163)

### Helpers: handle mapping
Closest-point queries and vertex-handle selection.

In [69]:
def closest_point_handles(
    source_vertices: Float[np.ndarray, "N 3"],
    target: Mesh,
) -> Tuple[Int[np.ndarray, " N"], Float[np.ndarray, "N 3"], Float[np.ndarray, "N 3"]]:
    """Compute closest points on a target mesh for each source vertex.

    Parameters
    ----------
    source_vertices
        Source vertices $(N, 3)$.
    target
        Target mesh to query against.

    Returns
    -------
    face_indices
        Face index in the target mesh for each closest point.
    points
        Closest points on the target mesh surface.
    barycentric
        Barycentric coordinates of the closest points within their faces.
    """
    _, face_indices, points = igl.point_mesh_squared_distance(
        source_vertices, target.vertices, target.faces
    )
    tri = target.faces[face_indices]
    v0 = target.vertices[tri[:, 0]]
    v1 = target.vertices[tri[:, 1]]
    v2 = target.vertices[tri[:, 2]]
    barycentric = igl.barycentric_coordinates(points, v0, v1, v2)
    return face_indices, points, barycentric


def vertex_handle_indices(
    face_indices: Int[np.ndarray, " N"],
    barycentric: Float[np.ndarray, "N 3"],
    faces: Int[np.ndarray, "F 3"],
) -> Int[np.ndarray, " N"]:
    """Pick a single face vertex as the handle for each closest point.

    Parameters
    ----------
    face_indices
        Face indices for each closest point.
    barycentric
        Barycentric weights per closest point.
    faces
        Target mesh faces.

    Returns
    -------
    handle_vertices
        Selected target vertex index per handle (max barycentric weight).
    """
    tri = faces[face_indices]
    max_idx = np.argmax(barycentric, axis=1)
    handle_vertices = tri[np.arange(tri.shape[0]), max_idx]
    return handle_vertices


In [70]:
# test these functions

face_indices, points, barycentric = closest_point_handles(source_vertices=mesh_low.vertices, target=mesh_high)

face_indices.max() > mesh_low.faces.max() 

np.True_

In [71]:
handle_vertices = vertex_handle_indices(face_indices, barycentric, mesh_high.faces)

(mesh_low.vertices.shape[0] <handle_vertices.max() < mesh_high.vertices.shape[0],
 handle_vertices.shape[0] == mesh_low.vertices.shape[0])

(np.True_, True)

In [29]:
# handles are basically unique. No two low res vertices map to the same high res vertex

handle_vertices.shape, np.unique(handle_vertices).shape 

((2309,), (2309,))

### Helpers: constraints + biharmonic solve
Aggregate handle constraints and solve the biharmonic displacement field.

In [33]:
def aggregate_vertex_constraints(
    n_vertices: int,
    handle_vertices: Int[np.ndarray, " N"],
    handle_disp: Float[np.ndarray, "N 3"],
) -> Tuple[Int[np.ndarray, " B"], Float[np.ndarray, "B 3"]]:
    """Average handle displacements for possibly repeated vertices.

    Parameters
    ----------
    n_vertices
        Number of vertices in the target mesh.
    handle_vertices
        Vertex index for each handle.
    handle_disp
        Displacement for each handle.

    Returns
    -------
    b
        Unique constrained vertex indices.
    bc
        Displacements at constrained vertices.
    """
    accum = np.zeros((n_vertices, 3), dtype=handle_disp.dtype)
    counts = np.zeros((n_vertices, 1), dtype=np.int32)
    accum[handle_vertices] += handle_disp
    counts[handle_vertices] += 1
    b = np.where(counts[:, 0] > 0)[0]
    bc = accum[b] / counts[b]
    return b, bc


def face_to_vertex_constraints(
    n_vertices: int,
    face_indices: Int[np.ndarray, " N"],
    faces: Int[np.ndarray, "F 3"],
    handle_disp: Float[np.ndarray, "N 3"],
) -> Tuple[Int[np.ndarray, " B"], Float[np.ndarray, "B 3"]]:
    """Convert face-based handles to vertex constraints by averaging.

    Each handle displacement is applied to the three face vertices and
    averaged per vertex. This is a simple approximation that preserves
    the handle point's displacement in expectation.

    Parameters
    ----------
    n_vertices
        Number of vertices in the target mesh.
    face_indices
        Face index for each handle.
    faces
        Target mesh faces.
    handle_disp
        Displacement for each handle.

    Returns
    -------
    b
        Unique constrained vertex indices.
    bc
        Displacements at constrained vertices.
    """
    tri = faces[face_indices]
    accum = np.zeros((n_vertices, 3), dtype=handle_disp.dtype)
    counts = np.zeros((n_vertices, 1), dtype=np.int32)
    for k in range(3):
        verts = tri[:, k]
        accum[verts] += handle_disp
        counts[verts] += 1
    b = np.where(counts[:, 0] > 0)[0]
    bc = accum[b] / counts[b]
    return b, bc


def solve_biharmonic(
    mesh: Mesh,
    b: Int[np.ndarray, " B"],
    bc: Float[np.ndarray, "B 3"],
    k: int = 2,
) -> Float[np.ndarray, "V 3"]:
    """Solve a biharmonic displacement field with boundary constraints.

    Parameters
    ----------
    mesh
        Target mesh.
    b
        Constrained vertex indices.
    bc
        Displacement constraints at indices in `b`.
    k
        Harmonic order (2 gives biharmonic deformation).

    Returns
    -------
    disp
        Displacement at each target vertex.
    """
    disp = igl.harmonic(mesh.vertices, mesh.faces, b, bc, k)
    return disp


In [47]:
### Utility function checks - simple sanity checks on the test data.

# let's do the registration first

face_indices, points, barycentric = closest_point_handles(source_vertices=mesh_low.vertices, target=mesh_high)
handle_vertices = vertex_handle_indices(face_indices, barycentric, mesh_high.faces)


In [48]:
# displacements from low-res edit
handle_disp = mesh_low_def.vertices - mesh_low.vertices

b_v, bc_v = aggregate_vertex_constraints(
    n_vertices=mesh_high.vertices.shape[0],
    handle_vertices=handle_vertices,
    handle_disp=handle_disp,
)

(b_v.shape, bc_v.shape, np.all(np.isfinite(bc_v)))

((2309,), (2309, 3), np.True_)

In [37]:
b_f, bc_f = face_to_vertex_constraints(
    n_vertices=mesh_high.vertices.shape[0],
    face_indices=face_indices,
    faces=mesh_high.faces,
    handle_disp=handle_disp,
)

(b_f.shape, bc_f.shape, np.all(np.isfinite(bc_f)))

((6916,), (6916, 3), np.True_)

In [46]:
%%time

disp_v = solve_biharmonic(mesh_high, b_v, bc_v, k=2) # 5s for k=1, 44 s for k=2
disp_f = solve_biharmonic(mesh_high, b_f, bc_f, k=2) # 5s for k=1, 40s for k=2
 
(disp_v.shape, disp_f.shape, np.isfinite(disp_v).all(), np.isfinite(disp_f).all())

CPU times: user 1min 24s, sys: 2.05 s, total: 1min 26s
Wall time: 1min 26s


((972454, 3), (972454, 3), np.True_, np.True_)

In [49]:
err_v = np.linalg.norm(disp_v[b_v] - bc_v, axis=1)
err_f = np.linalg.norm(disp_f[b_f] - bc_f, axis=1)

(err_v.max(), err_f.max())

(np.float64(0.0), np.float64(0.0))

### Helpers: transfer pipelines
Wrap the steps for reuse in the demos.

In [67]:
def transfer_with_vertex_handles(
    mesh_low: Mesh,
    mesh_low_def: Mesh,
    mesh_high: Mesh,
) -> Tuple[Mesh, Float[np.ndarray, "V 3"], Int[np.ndarray, " N"], Float[np.ndarray, "N 3"]]:
    """Transfer deformation using vertex handles.

    Parameters
    ----------
    mesh_low
        Original low-res mesh $B$.
    mesh_low_def
        Deformed low-res mesh $C$.
    mesh_high
        High-res mesh $A$.

    Returns
    -------
    mesh_high_def
        Deformed high-res mesh.
    disp
        Per-vertex displacement on the high-res mesh.
    handle_vertices
        Vertex handle indices on the high-res mesh.
    handle_disp
        Displacement at each handle.
    """
    face_indices, _, barycentric = closest_point_handles(
        mesh_low.vertices, mesh_high
    )
    handle_vertices = vertex_handle_indices(
        face_indices, barycentric, mesh_high.faces
    )
    handle_disp = mesh_low_def.vertices - mesh_low.vertices
    b, bc = aggregate_vertex_constraints(
        mesh_high.vertices.shape[0], handle_vertices, handle_disp
    )
    disp = solve_biharmonic(mesh_high, b, bc, k=2)
    mesh_high_def = Mesh(mesh_high.vertices + disp, mesh_high.faces)
    return mesh_high_def, disp, handle_vertices, handle_disp


def transfer_with_barycentric_handles(
    mesh_low: Mesh,
    mesh_low_def: Mesh,
    mesh_high: Mesh,
) -> Tuple[Mesh, Float[np.ndarray, "V 3"], Int[np.ndarray, " N"], Float[np.ndarray, "N 3"], Float[np.ndarray, "N 3"]]:
    """Transfer deformation using barycentric face handles.

    Parameters
    ----------
    mesh_low
        Original low-res mesh $B$.
    mesh_low_def
        Deformed low-res mesh $C$.
    mesh_high
        High-res mesh $A$.

    Returns
    -------
    mesh_high_def
        Deformed high-res mesh.
    disp
        Per-vertex displacement on the high-res mesh.
    face_indices
        Face index for each handle on the high-res mesh.
    handle_disp
        Displacement at each handle.
    barycentric
        Barycentric weights for each handle.
    """
    face_indices, _, barycentric = closest_point_handles(
        mesh_low.vertices, mesh_high
    )
    handle_disp = mesh_low_def.vertices - mesh_low.vertices
    b, bc = face_to_vertex_constraints(
        mesh_high.vertices.shape[0], face_indices, mesh_high.faces, handle_disp
    )
    disp = solve_biharmonic(mesh_high, b, bc, k=2) # k =2
    mesh_high_def = Mesh(mesh_high.vertices + disp, mesh_high.faces)
    return mesh_high_def, disp, face_indices, handle_disp, barycentric


### Helpers: synthetic deformation
A small nonlinear deformation for testing.

In [61]:
def make_synthetic_deformation(
    vertices: Float[np.ndarray, "V 3"],
) -> Float[np.ndarray, "V 3"]:
    """Create a smooth synthetic deformation for testing.

    Parameters
    ----------
    vertices
        Input vertex positions.

    Returns
    -------
    vertices_def
        Deformed vertex positions.
    """
    center = vertices.mean(axis=0)
    r = np.linalg.norm(vertices - center, axis=1)
    r_max = r.max() + 1e-8
    bump = 0.25 * np.exp(-(r**2) / (0.15 * r_max**2))
    span_y = np.ptp(vertices[:, 1]) + 1e-8
    twist = 0.15 * np.sin(2 * np.pi * (vertices[:, 1] - vertices[:, 1].min()) / span_y)
    d = np.stack([0.05 + 0.1 * twist, 0.0 * twist, bump], axis=1)
    vertices_def = vertices + d
    return vertices_def


### Phase 1: vertex handles (simple shift)
Use the current `mesh_low_def` shift to test the simplest pipeline.

In [62]:
mesh_high_def, disp, handle_vertices, handle_disp = transfer_with_vertex_handles(
    mesh_low, mesh_low_def, mesh_high
)
pred_disp = disp[handle_vertices]
err = np.linalg.norm(pred_disp - handle_disp, axis=1)
print("vertex handles | mean err:", err.mean(), "max err:", err.max())
meshplot.plot(mesh_high_def.vertices, mesh_high_def.faces)


vertex handles | mean err: 0.0 max err: 0.0


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0601241…

### Phase 1: vertex handles (synthetic deformation)
Replace the mock shift with a smooth, nonlinear deformation and repeat.

In [63]:
mesh_low_def = Mesh(
    make_synthetic_deformation(mesh_low.vertices),
    mesh_low.faces.copy(),
)

In [64]:
mesh_high_def, disp, handle_vertices, handle_disp = transfer_with_vertex_handles(
    mesh_low, mesh_low_def, mesh_high
)
pred_disp = disp[handle_vertices]
err = np.linalg.norm(pred_disp - handle_disp, axis=1)
print("vertex handles (synthetic) | mean err:", err.mean(), "max err:", err.max())
meshplot.plot(mesh_high_def.vertices, mesh_high_def.faces)


vertex handles (synthetic) | mean err: 0.0 max err: 0.0


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0601241…

### Phase 2: barycentric handles
Use face handles and barycentric weights for a closer geometric constraint.

In [65]:
mesh_high_def_bary, disp_bary, face_indices, handle_disp, bary = transfer_with_barycentric_handles(
    mesh_low, mesh_low_def, mesh_high
)
tri = mesh_high.faces[face_indices]
disp0 = disp_bary[tri[:, 0]]
disp1 = disp_bary[tri[:, 1]]
disp2 = disp_bary[tri[:, 2]]
pred_disp = (
    bary[:, [0]] * disp0 + bary[:, [1]] * disp1 + bary[:, [2]] * disp2
)
err = np.linalg.norm(pred_disp - handle_disp, axis=1)
print("barycentric handles | mean err:", err.mean(), "max err:", err.max())
meshplot.plot(mesh_high_def_bary.vertices, mesh_high_def_bary.faces)


barycentric handles | mean err: 1.7003406187977287e-06 max err: 0.0007194708576546011


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0601198…

### Meshplot in VS Code (widget issue)
`meshplot` uses ipywidgets + pythreejs. In VS Code, the widget frontend may not load `jupyter-threejs`, causing the RendererModel error.
If the widget still fails after restarting the kernel, use the export cell below and visualize the results in Blender instead.

In [ ]:
export_dir = "exports"
os.makedirs(export_dir, exist_ok=True)

#igl.writeOBJ(os.path.join(export_dir, "mesh_low.obj"), mesh_low.vertices, mesh_low.faces)
#igl.writeOBJ(os.path.join(export_dir, "mesh_low_def_synthetic.obj"), mesh_low_def.vertices, mesh_low_def.faces)
#igl.writeOBJ(os.path.join(export_dir, "mesh_high.obj"), mesh_high.vertices, mesh_high.faces)

igl.writeOBJ(os.path.join(export_dir, "mesh_high_def_vertex.obj"), mesh_high_def.vertices, mesh_high_def.faces)
igl.writeOBJ(
    os.path.join(export_dir, "mesh_high_def_bary.obj"),
    mesh_high_def_bary.vertices,
    mesh_high_def_bary.faces,
 )
print(f"Wrote OBJ files to {export_dir}/")


Wrote OBJ files to exports/
